# ***Importing necessary classes and modules***

In [1]:
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string



In [2]:
!pip install opendatasets

In [3]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4227147 sha256=ed468131596a97476741092533e759e0c3a8be8fd1773e714be9354661ba608c
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [4]:
import opendatasets as od
# import kaggle
import pandas as pd
import numpy as np
import string
import nltk
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import fasttext
import sklearn

In [5]:
nltk.download("wordnet")
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# ***Crawling the data***

In this notebook we are going to crawl **the reviews for The Godfather series( The Godfather, The Godfather Part II, The Godfather PartIII) from Letterboxd and then anlyze what people feel about each of them and rank the results.**

همانطور که در سل بالا گفته شد در این تمرین ابتدا نظرات کاربران برای مجموعه فیلم های پدرخوانده را از سایت لترباکسد کراول میکنیم.
نظرات کراول شده را پیش پردازش میکنیم.
بعد از آن تسکی که برای این مجموعه نظرات تعریف کرده ایم آن است که به تحلیل نظرات و احساسات مردم نسبت به این 3 فیلم بپردازیم و آنها را بر این اساس رتبه بندی کنیم.

First we import requests and BeautifulSoup modules which will be used in crawling the reviews for films.

In [6]:
import requests
from bs4 import BeautifulSoup



Now we implement a function to crawl a single page of reviews, keep in mind that in letterboxd each page contains 12 reviews and we crawl the reviews of 250 pages for each of these 3 movies, so we have 3000 reviews for each movie and 9000 in total.

the **crawl_first_page** function we want to crawl a single page of letterboxd reveiws, in order to do that we get the text of the page using **request** and **beautifulsoup** and after that we extract the data with **class = 'body-text -prose collapsible-text', which the text of all the reviews** and return the text of all the reviews of that page.

after that we need a function to generate all the urls for each of the 250 pages of each movie, so we define **generate_url** function which takes a url pattern of the pages and the number of pages we want to crawl and returns list of those urls.

Now that we have the two functions **crawl_first_page** and **generate_url**, we can define the function **crawl_all** that uses these two to crawl all the reviews of all the pages.

In [7]:

# function to crawl a single page of reveiws
def crawl_first_page(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  # reviews = soup.find_all(class_='film-detail-body-text')
  # print(soup.prettify())

  # <li class="film-detail"
  # tp = soup.find_all(class_="film-detail")
  # print(tp)
  # for review in soup.find_all(class_="film-detail"):
    # print(review.get('href'))

  tp = soup.find_all(class_='body-text -prose collapsible-text')
  # print(tp)
  cntr = 1
  reviews = []
  for review in tp:
    # print(cntr, review.get_text())
    # cntr +=1
    reviews.append(review.get_text())
  return reviews

def generate_url(url, url_pattern, count):
  url_splt = url_pattern.split('page/')
  # print(url_splt)
  pattern = url_splt[0]
  urls = []
  urls.append(url)
  for i in range(1, count):
    idx = i+1
    url_idx = pattern + f"page/{idx}/"
    urls.append(url_idx)

  return urls

def crawl_all(urls):
  all_reviews = []
  all_reviews_splt = []
  for url in urls:
    reviews = crawl_first_page(url)
    all_reviews.append(reviews)
    for rev in reviews:
      all_reviews_splt.append(rev)

  return all_reviews_splt





Now we use urls for each movie and crawl the data, then print the crawled data and also save it into a txt file.

In [25]:

# url_pattern = "https://letterboxd.com/film/dune-part-two/reviews/by/activity/page/2/"
count = 250 # max for inception
# url = "https://letterboxd.com/film/dune-part-two/reviews/by/activity/"
# url = "https://letterboxd.com/film/inception/reviews/by/activity/"
# url_pattern = "https://letterboxd.com/film/inception/reviews/by/activity/page/2/"
godfather_url = "https://letterboxd.com/film/the-godfather/reviews/by/activity/"
godfather_url_pattern = "https://letterboxd.com/film/the-godfather/reviews/by/activity/page/250/"
godfather2_url = "https://letterboxd.com/film/the-godfather-part-ii/reviews/by/activity/"
godfather2_url_pattern = "https://letterboxd.com/film/the-godfather-part-ii/reviews/by/activity/page/250/"
godfather3_url = "https://letterboxd.com/film/the-godfather-part-iii/reviews/by/activity/"
godfather3_url_pattern = "https://letterboxd.com/film/the-godfather-part-iii/reviews/by/activity/page/250/"

urls_gf1 = generate_url(godfather_url, godfather_url_pattern, count)
urls_gf2 = generate_url(godfather2_url, godfather2_url_pattern, count)
urls_gf3 = generate_url(godfather3_url, godfather3_url_pattern, count)

# print(urls)
urls = urls_gf1 + urls_gf2 + urls_gf3

# crawl_first_page(url)
all_reviews = crawl_all(urls)
cntr = 1
all_reviews_enumarated = []
for review in all_reviews:
  review = f"{cntr}" + review
  print(review)
  all_reviews_enumarated.append(review)
  # print(cntr, review)
  cntr += 1

# reviews = crawl_letterboxd_reviews(url_pattern, num_pages)
# print(reviews)
# Print the crawled reviews in text format
# for i, review in enumerate(reviews, start=1):
#     print(f"Review {i}:")
#     print(review)
#     print()


Streaming output truncated to the last 5000 lines.
4001 De Niro > Brando 98/100 
4002  Well here it is. After months in anticipation of my 500th movie. Months of anticipation and delays, here it is. My 500th movie: The Godfather Part II. “I Don’t feel I have to wipe everybody out, Tom. Just My Enemies.”A prequel and a sequel to Francis Ford Coppola’s 1972 groundbreaking Mob film “The Godfather,” “The Godfather Part II” continues the story of the Corleone Family in 2 ways. One part is a prequel, focusing on a young Vito and…  
4003 Yeh The Godfather Part 1 is so much better in my opinion 
4004 al pacino 
4005 "I don't feel I have to wipe everybody out, Tom. Just my enemies, that's all."There is a wonderful contrast here between Vito's optimism for the family under the influence of the American Dream and Michael turning the family into a business, leading to what is essentially his and the family's utter downfall. One of the most depressing character arcs of all time.Prefer the first fil

In [26]:
# Print and save preprocessed reviews to a text file
output_filename = "godfather_reviews.txt"
with open(output_filename, "w", encoding="utf-8") as output_file:
    for i, review in enumerate(all_reviews_enumarated, start=1):
        output_file.write(review + "\n")

print("Reviews saved to:", output_filename)

Reviews saved to: godfather_reviews.txt


همانطور که در بالا میبینید با موفقیت 9000 ریویو را کراول کردیم.

حالا چند کتابخانه برای پیش پردازش این ریویو ها ایمپورت میکنیم.

In [13]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=a921523d15b55cf98fb4a92a004ab8ca04b4632c2994c29106312562fc9de345
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [14]:
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 3.4 MB/s eta 0:00:00


In [15]:
!pip install autocorrect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622363 sha256=214fa194f5244654e2e07e321f5284b583cb6489516111569a7e41bd2b72150d
  Stored in directory: /root/.cache/pip/wheels/b5/7b/6d/b76b29ce11ff8e2521c8c7dd0e5bfee4fb1789d76193124343
Successfully built autocorrect


In [16]:
import spacy
import nltk
from typing import Optional
import string
import re
from langdetect import detect
import emoji
from autocorrect import Speller





In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ***Preprocess The Reviews***

In this secrtion we perform preprocessing on all the reviews.
Preprocessing is a crucial step in natural language processing (NLP) and text mining tasks. It involves cleaning and transforming raw text data into a format that is suitable for analysis and modeling. The goal of preprocessing is to enhance the quality of the text data, making it more manageable and facilitating more accurate analysis and interpretation.

Here's an overview of preprocessing, including its most commonly used methods and its importance:

Tokenization: Tokenization is the process of breaking down text into smaller units, typically words or tokens. This step makes it easier to analyze and manipulate the text data. Common tokenization methods include splitting text based on whitespace or punctuation.

Normalization: Normalization involves transforming text into a standardized format. This may include converting all text to lowercase, removing accents, or expanding contractions (e.g., "can't" to "cannot"). Normalization helps ensure consistency in the text data and reduces the complexity of analysis.

Removing Noise: Noise refers to irrelevant or unwanted elements in the text data, such as HTML tags, special characters, or non-textual content (e.g., URLs, emojis). Removing noise helps improve the quality of the text and prevents it from interfering with analysis.

Removing Stopwords: Stopwords are common words that often do not carry significant meaning in text analysis, such as "the," "is," and "and." Removing stopwords can reduce the dimensionality of the text data and improve the efficiency of analysis by focusing on more meaningful terms.

Stemming and Lemmatization: Stemming and lemmatization are techniques for reducing words to their root or base form. Stemming involves removing prefixes and suffixes from words to extract their stem, while lemmatization involves mapping words to their dictionary form (lemma). These techniques help consolidate variations of words and improve the accuracy of analysis.

Spell Checking: Spell checking involves identifying and correcting spelling errors in the text data. This step helps ensure the accuracy and consistency of the text, which is particularly important for tasks such as sentiment analysis and information retrieval.

In [18]:
!pip install symspellpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for editdistpy: filename=editdistpy-0.1.3-cp310-cp310-linux_x86_64.whl size=187478 sha256=9b829113b68ece028c52a21be5990d6a02e0498fb364fea8ba30702ed2d82a85
  Stored in directory: /root/.cache/pip/wheels/88/6a/a6/a1283cc145323a1fb3d475bd158ee60b248ab1985230d266fc
Successfully built editdistpy


In [19]:
!pip install spellchecker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for spellchecker: filename=spellchecker-0.4-py3-none-any.whl size=3966500 sha256=0523ffa8ca67899a0c5a6fa8c50c1e20f8c8b49180a53ae48a27d7304764bf25
  Stored in directory: /root/.cache/pip/wheels/6c/90/c3/eac248d8755b2a7343487a2087b4b29ad98f388c3c8c69c286
  Created wheel for inexactsearch: filename=inexactsearch-1.0.2-py3-none-any.whl size=7123 sha256=eb5e9214da15e577f4f4ac8310ab226689612a5e7d8ac3b5b4dc13566762320c
  Stored in directory: /root/.cache/pip/wheels/63/19/2c/5e9f447f2533d457a1167c3e553f235e232b8a639e3f5fafab
  Created wheel for silpa_common: filename=silpa_common-0.3-py3-none-any.whl size=8466 sha256=a6fda216b4f399116c4677ba765f5c4d5097df3d5e803b605973ca068c63d75d
  Stored in directory: /root/.cache/pip/wheels/c0/72/43/0c7

بعد از بررسی ریویو هایی که از سایت لترباکسد کراول کردیم، متوجه میشویم تعداد زیادی از آنها شامل غلط املایی میشوند، همچنین تعداد بالایی از آنها هم شامل ایموجی هستند، بغیر از این موارد تعداد قابل توجهی از آنها به زبانی غیر از اینگلیسی نوشته شده بودند، از این رو توجه به حذف و تصحیح این سه مورد در پیش پردازش آنها از اهمیت بالایی برخوردار است.

In [20]:
import re
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from typing import Optional
from spacy import load as spacy_load
from autocorrect import Speller

class Preprocessor:
    instance: Optional['Preprocessor'] = None

    @staticmethod
    def get_instance(stopwords_path= None):
        if Preprocessor.instance == None:
            Preprocessor.instance = Preprocessor(stopwords_path)
        return Preprocessor.instance

    def __init__(self, stopwords_path):
        # Create a variable of stop words.
        self.stopwords = set(nltk.corpus.stopwords.words('english'))
        # Load the spacy model.
        self.nlp = spacy_load('en_core_web_sm')
        # Create a spell checker
        self.speller = Speller(lang='en')
        # self.speller.load_dictionary("frequency_dictionary_en_82_765.txt", 0, 1)


    def preprocess(self, text):
        # The main function of the class.
        text = self.remove_links(text)
        # text = self.expand_contractions(text)
        text = self.normalize(text)
        text = self.correct_spelling(text)
        # print(text)
        words = self.word_tokenize(text)
        # print(words)
        words = self.remove_stopwords(words)
        return words

    def normalize(self, text):
        # Normalize text (lower case, remove punctuations, lemmatization)
        text = text.lower()
        text = self.remove_punctuations(text)
        text = self.expand_contractions(text)
        text = self.lemmatize(text)
        return text

    def lemmatize(self, text):
        # Lemmatize text
        doc = self.nlp(text)
        # print(doc)
        negation_words = ['not', 'no', 'never', 'none', 'neither', 'nor', 'nobody', 'nowhere', 'nothing']
        # accept_words = [word for word in text if word.lower() not in self.stopwords or word.lower() in negation_words]
        text = ' '.join([token.lemma_ for token in doc if token.lemma_ not in p.stopwords or token.lemma_ in negation_words])
        return text

    def remove_links(self, text):
        # Remove links
        text = re.sub(r'http\S+', '', text)
        return text

    def remove_punctuations(self, text):
        # Remove punctuations
        text = text.translate(str.maketrans('', '', string.punctuation))
        return text

    def word_tokenize(self, text):
        # Tokenize text
        words = nltk.word_tokenize(text)
        return words

    def remove_stopwords(self, words):
        # Remove stopwords
        negation_words = ['not', 'no', 'never', 'none', 'neither', 'nor', 'nobody', 'nowhere', 'nothing']

        words = [word for word in words if word.lower() not in self.stopwords or word.lower() in negation_words]
        return words

    def expand_contractions(self, text):
        # Expand contractions like "isn't", "aren't", etc.
        contractions_dict = {
            "isn't": "is not",
            "isnt": "is not",
            "aren't": "are not",
            "arent": "are not",
            "wasn't": "was not",
            "wasnt": "was not",
            "weren't": "were not",
            "werent": "were not",
            "didn't": "did not",
            "didnt": "did not",
            "thi ": "this ",
            "ain't": "am not",
            "aint": "am not",
            "can't": "cannot",
            "couldn't": "could not",
            "doesn't": "does not",
            "don't": "do not",
            "hadn't": "had not",
            "hasn't": "has not",
            "haven't": "have not",
            "it's": "it is",
            "let's": "let us",
            "mustn't": "must not",
            "shan't": "shall not",
            "shouldn't": "should not",
            "won't": "will not",
            "wouldn't": "would not"
            # Add more contractions as needed
        }
        pattern = re.compile(r'\b(' + '|'.join(contractions_dict.keys()) + r')\b')
        expanded_text = pattern.sub(lambda match: contractions_dict[match.group(0)], text)
        return expanded_text

    def remove_emojis(self, text):
        # Remove emojis using regex
        text = re.sub(r'[\U00010000-\U0010ffff]', '', text)
        return text

    def remove_non_english_text(self, text):
        # Remove non-English text
        try:
            if detect(text) != 'en':
                return ''
        except:
            pass
        return text

    def correct_spelling(self, text):
        # Correct spelling errors
        corrected_text = []
        words = word_tokenize(text)
        for word in words:
            corrected_word = self.speller(word)
            corrected_text.append(corrected_word)
        return ' '.join(corrected_text)

In the preprocess class we define a function for normalizing( normalize ) in which we convert the text into lowercase, remove punctuations, expand contractions, and in the end we lemmatize the text. In **lemmaize** function we use **spacy_load('en_core_web_sm')** to get the lemma for each token. We also define a function for removing links, because some of the reviews contain links. In **remove_punctuations** we remove the punctuations using **string.punctuation**. I implemented a function to tokenize the text using **nltk.word_tokenize**. I used **nltk.corpus.stopwords.words('english')** to remove stopwords but made sure to **keep the negation words as they are important to our task**. We also have a function for removing emojis and also use **Speller** from **autocorrect** to correct the spelling.

In [21]:
# Example usage:
p = Preprocessor.get_instance(None)
text = "Thi isnt a testi 😭🥺 I am hapy"
text = text.lower()
text = p.expand_contractions(text)
text = p.correct_spelling(text)
text = p.remove_emojis(text)
text

'this is not a test  i am happy'

In [27]:
txt = " 😭🥺 این انگلیسی نیست هست؟"
txt = p.remove_non_english_text(txt)
txt = p.correct_spelling(txt)
txt = p.expand_contractions(txt)
txt = p.remove_emojis(txt)
txt

''

In [28]:
ex = "the mvie wasnt that amzing"
ex = p.preprocess(ex)
print(ex)

['movie', 'not', 'amazing']


همان طور که مثال های بالا نشان میدهد متن به خوبی پیش پردازش میشود.

دقت کنید که چون تعداد خیلی زیادی از ریویو ها شکل صحیح نگارش مخفف های و کوتاه سازی های نوشتاری افعال و سایر موارد را رعایت نکرده اند و در موارد زیادی از گذاشتن ' در نوشتار خود در مخفف سازی افعال استفاده نکرده اند اهمیت قابل توجهی برای تصحیح آنها وجود داشت که برای این منظور تابعی در کلاس پیش پردازشمان تعریف کرده ایم.
همچنین توجه کنید که چون در ادامه میخواهیم به بررسی نظر مثبت یا منفی مردم نسبت به هر یک از این سه فیلم بپردازیم، از عبارات نفی کننده در استاپ وردز یا کلمات زاعد استفاده نکردیم.

Note that because a high number of the reviews contain a bad form of writing, for example they often write "i didnt like this movie because it wasnt that good" instead of "i didn't like this movie because it wasn't that good", i implemented the function **expand_contractions** in the preprocess class.
** *Also because the task we define is to analyze people's emotions about these three movies we shouldn't consider negation words such as "not", as a stopword( for example suppose we have a review saying "this movie was not good", if we take "not" as a stopword and remove it at the beginning the preprocessed version of this review will be "this movie was good", which is the opposite of what the reviewer wanted to express.) so I define a list of negation words to keep and not have them be removed when removing stopwords.**

In [30]:
# Read the text from the file
filename = "godfather_reviews.txt"
with open(filename, "r", encoding="utf-8") as file:
    reviews = file.readlines()

# Preprocess each review
preprocessed_reviews = []
for review in reviews:
    # Extract the review text (after the number)
    review_text = review.split(maxsplit=1)[1]

    # Remove emojis, non-English text, and correct spelling errors
    review_text = p.remove_non_english_text(review_text)
    review_text = p.correct_spelling(review_text)
    review_text = p.expand_contractions(review_text)
    review_text = p.remove_emojis(review_text)

    # review_text = p.correct_spelling(review_text)

    # Perform the rest of the preprocessing
    preprocessed_review = p.preprocess(review_text)

    # Join the preprocessed words back into a single string
    preprocessed_review = " ".join(preprocessed_review)

    # Append the preprocessed review to the list
    preprocessed_reviews.append(preprocessed_review)

# # Print the preprocessed reviews
# for i, review in enumerate(preprocessed_reviews, start=1):
#     print(f"Preprocessed Review {i}:")
#     print(review)
#     print()

In [31]:
# Print and save preprocessed reviews to a text file
output_filename = "preprocessed_reviews.txt"
with open(output_filename, "w", encoding="utf-8") as output_file:
    for i, review in enumerate(preprocessed_reviews, start=1):
        print(f"Preprocessed Review {i}:")
        print(review)
        print()
        output_file.write(review + "\n")

print("Preprocessed reviews saved to:", output_filename)

Streaming output truncated to the last 5000 lines.

Preprocessed Review 7335:
coda cut live utter perfection first two entry might not ever truly possible part ii always feel like epilogue cutting fc ' final cut film appropriately dub coda fix many problem plague theatrical cut wisely excuse `` previously `` tighten pacing cut sofia cupola ' performance highlight scene actress respect ' due ' great …

Preprocessed Review 7336:


Preprocessed Review 7337:
watch godfather coda death michael corleone honest long since watch original cut not totally sure difference — though new opening scene go long well help viewer understand one strange plotting film international immobilare probably 15 year since see would college memory always not bad …

Preprocessed Review 7338:
finish watch godfather movie ' speech good series movie ever watch entire life

Preprocessed Review 7339:
godfather 3 really one bad sequel alltime godfather coda death michael corleone single great director cut salvage job fi

در سل بالا تمامی ریویو هارا پیش پردازش کردیم و علاوه بر پرینت کردن در فایل تکستی ذخیره میکنیم.

# ***TF-IDF score to extract keywords.***

TF-IDF, short for Term Frequency-Inverse Document Frequency, is a statistical measure used to evaluate the importance of a word within a document relative to a collection of documents (corpus). It is commonly used in information retrieval and text mining to determine the significance of a term in a document.

Now we use TF-IDF method to extract Keywords from the preprocessed reviews.

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=1000)

# Fit and transform the preprocessed reviews
tfidf_matrix = tfidf_vectorizer.fit_transform(preprocessed_reviews)



In [33]:
# Get feature names (keywords)
keywords = tfidf_vectorizer.get_feature_names_out()


In [34]:
print(keywords)

['10' '100' '1010' '12' '15' '1972' '1974' '1990' '1st' '20' '2020' '2nd'
 '30' '35' '3rd' '40' '45' '4k' '50' '50th' '70' '90' 'able' 'abortion'
 'absolute' 'absolutely' 'academy' 'acclaim' 'achievement' 'across' 'act'
 'acting' 'action' 'actor' 'actress' 'actual' 'actually' 'add' 'addition'
 'admit' 'age' 'ago' 'agree' 'al' 'allow' 'alltime' 'almost' 'alone'
 'along' 'already' 'alright' 'also' 'although' 'always' 'amazing'
 'ambitious' 'america' 'american' 'amount' 'andy' 'anniversary' 'another'
 'answer' 'anymore' 'anyone' 'anything' 'anyway' 'apart' 'apocalypse'
 'appear' 'appreciate' 'approach' 'arc' 'arguably' 'argue' 'around' 'art'
 'aside' 'ask' 'aspect' 'ass' 'attempt' 'attention' 'audience' 'auto'
 'award' 'away' 'awesome' 'awful' 'baby' 'back' 'backstory' 'bad'
 'baptism' 'bar' 'barely' 'base' 'basically' 'bc' 'bear' 'beat'
 'beautiful' 'beautifully' 'become' 'bed' 'begin' 'beginning' 'behind'
 'believe' 'besides' 'betrayal' 'beyond' 'big' 'birthday' 'bit' 'black'
 'blood' '

As you can see above not the keywords are extracted based on their TF-IDF score, meaning they are selected based on their relevance in the reviews. Notcie that not every extracted keyword is a positive concept, so the score that we will calculate in the next cell will tell us how passionate people were about each movie, note that not all passion is good passion :)  

In [35]:

# Calculate the sum of TF-IDF scores for each keyword across all reviews
keyword_scores = tfidf_matrix.sum(axis=0).A1

# Create a dictionary of keywords and their scores
keyword_scores_dict = {keyword: score for keyword, score in zip(keywords, keyword_scores)}

# Rank the keywords based on their scores
ranked_keywords = sorted(keyword_scores_dict.items(), key=lambda x: x[1], reverse=True)

# Print the top keywords
print("Top Keywords:")
for i, (keyword, score) in enumerate(ranked_keywords[:20], start=1):
    print(f"{i}. {keyword}: {score}")

Top Keywords:
1. film: 387.1395322215178
2. not: 386.6420872473869
3. movie: 348.8403293983857
4. godfather: 324.52779260322137
5. one: 272.21050529389015
6. good: 262.53612611681996
7. watch: 261.1232289512848
8. michael: 251.1738052285872
9. like: 244.68958560816793
10. part: 242.9447661882994
11. first: 241.60802565709744
12. time: 216.16486214677985
13. corleone: 208.50618932793543
14. make: 199.54276243589885
15. see: 197.16154875965228
16. say: 189.72362975832866
17. great: 189.62025221600453
18. well: 180.49958229996065
19. think: 174.20944837526562
20. cupola: 172.42278359211446


In [36]:
# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=1000)

# Fit and transform the preprocessed reviews for each movie segment
tfidf_matrix_1 = tfidf_vectorizer.fit_transform(preprocessed_reviews[:3000])  # Godfather
tfidf_matrix_2 = tfidf_vectorizer.fit_transform(preprocessed_reviews[3000:6000])  # Godfather Part II
tfidf_matrix_3 = tfidf_vectorizer.fit_transform(preprocessed_reviews[6000:])  # Godfather Part III

# Calculate the sum of TF-IDF scores for each movie segment
tfidf_scores_1 = tfidf_matrix_1.sum(axis=0).A1
tfidf_scores_2 = tfidf_matrix_2.sum(axis=0).A1
tfidf_scores_3 = tfidf_matrix_3.sum(axis=0).A1

# Create dictionaries to store keyword scores for each movie
keyword_scores_1 = {keyword: score for keyword, score in zip(tfidf_vectorizer.get_feature_names_out(), tfidf_scores_1)}
keyword_scores_2 = {keyword: score for keyword, score in zip(tfidf_vectorizer.get_feature_names_out(), tfidf_scores_2)}
keyword_scores_3 = {keyword: score for keyword, score in zip(tfidf_vectorizer.get_feature_names_out(), tfidf_scores_3)}

# Calculate the total TF-IDF score for each movie
total_tfidf_score_1 = sum(tfidf_scores_1)
total_tfidf_score_2 = sum(tfidf_scores_2)
total_tfidf_score_3 = sum(tfidf_scores_3)

# Rank the movies based on their total TF-IDF scores
ranked_movies_tfidf = [("Godfather", total_tfidf_score_1), ("Godfather Part II", total_tfidf_score_2), ("Godfather Part III", total_tfidf_score_3)]
ranked_movies_tfidf = sorted(ranked_movies_tfidf, key=lambda x: x[1], reverse=True)

# Print the ranked movies based on TF-IDF scores
print("Ranking based on TF-IDF keyword extraction:")
for i, (movie, score) in enumerate(ranked_movies_tfidf, start=1):
    print(f"{i}. {movie}: {score} total TF-IDF score")

Ranking based on TF-IDF keyword extraction:
1. Godfather Part II: 10151.412628878907 total TF-IDF score
2. Godfather Part III: 10036.444092897815 total TF-IDF score
3. Godfather: 9293.31283794207 total TF-IDF score


A higher TF-IDF (Term Frequency-Inverse Document Frequency) score for a term in a document indicates that the term is more important or relevant to that specific document compared to other documents in the corpus. TF-IDF is a statistical measure used in natural language processing to evaluate the significance of a term in a document relative to a collection of documents.


# ***Sentiment Analysis***

Now we use sentiment analysis method to rank the three movies based on their popularity.

In [37]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [38]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Initialize SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Calculate sentiment scores for each preprocessed review
sentiment_scores = []
for review in preprocessed_reviews:
    sentiment_score = sia.polarity_scores(review)
    sentiment_scores.append(sentiment_score)



In [39]:
# Calculate the overall sentiment score for each movie
movie_sentiment_scores = {"Godfather": 0, "Godfather Part II": 0, "Godfather Part III": 0}
for i, sentiment_score in enumerate(sentiment_scores):
    movie = "Godfather" if i < 3000 else ("Godfather Part II" if i < 6000 else "Godfather Part III")
    movie_sentiment_scores[movie] += sentiment_score['compound']

# Rank the movies based on their overall sentiment scores
ranked_movies_sentiment = sorted(movie_sentiment_scores.items(), key=lambda x: x[1], reverse=True)

# Print the ranked movies based on sentiment scores
print("Ranking based on sentiment analysis:")
for i, (movie, score) in enumerate(ranked_movies_sentiment, start=1):
    print(f"{i}. {movie}: {score} overall sentiment score")

Ranking based on sentiment analysis:
1. Godfather Part II: 1001.8576000000007 overall sentiment score
2. Godfather: 908.1538 overall sentiment score
3. Godfather Part III: 356.46340000000095 overall sentiment score


The results above show that comments on Godfather Part II, are most positive amongst these three, followed by Godfather, and finally Godfather Part III.

Which is interesting because based on their Letterboxd score people rated The Godfather Part II( 4.59 ) higher than The Godfather( 4.55 ), and they rated The Godfather higher than The Godfather Part III( 3.45 ).